#### Monday, February 10, 2025

Manually created this notebook from [Web Browser Automation with Agents](https://huggingface.co/docs/smolagents/examples/web_browser). The 'Open in Colab' button was pointing to a non-existent notebook.

[Web Browser Auomation with Agents github](https://github.com/huggingface/smolagents/blob/main/docs/source/en/examples/web_browser.md)

In [ ]:
# pip install smolagents selenium helium pillow -q

In [1]:
from io import BytesIO
from time import sleep

import helium
from dotenv import load_dotenv
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from smolagents import CodeAgent, tool
from smolagents.agents import ActionStep

# Load environment variables
load_dotenv()

False

In [2]:
@tool
def search_item_ctrl_f(text: str, nth_result: int = 1) -> str:
    """
    Searches for text on the current page via Ctrl + F and jumps to the nth occurrence.
    Args:
        text: The text to search for
        nth_result: Which occurrence to jump to (default: 1)
    """
    elements = driver.find_elements(By.XPATH, f"//*[contains(text(), '{text}')]")
    if nth_result > len(elements):
        raise Exception(f"Match n°{nth_result} not found (only {len(elements)} matches found)")
    result = f"Found {len(elements)} matches for '{text}'."
    elem = elements[nth_result - 1]
    driver.execute_script("arguments[0].scrollIntoView(true);", elem)
    result += f"Focused on element {nth_result} of {len(elements)}"
    return result

@tool
def go_back() -> None:
    """Goes back to previous page."""
    driver.back()

@tool
def close_popups() -> str:
    """
    Closes any visible modal or pop-up on the page. Use this to dismiss pop-up windows!
    This does not work on cookie consent banners.
    """
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()

In [ ]:
# Configure Chrome options
chrome_options = webdriver.ChromeOptions()

# I added these next 2 lines  because I want to use Brave browser ... I DO NOT WANT TO USE CHROME!
chrome_options.binary_location = "/usr/bin/brave-browser"
# enable if running this on the kitchen computer!
if false:
    chrome_options.binary_location = "/snap/bin/brave"
chrome_options.add_argument('--remote-debugging-port=9224') 

chrome_options.add_argument("--force-device-scale-factor=1")
chrome_options.add_argument("--window-size=1000,1350")
chrome_options.add_argument("--disable-pdf-viewer")
chrome_options.add_argument("--window-position=0,0")

In [9]:
# Initialize the browser
driver = helium.start_chrome(headless=False, options=chrome_options)

In [10]:
# Set up screenshot callback
def save_screenshot(memory_step: ActionStep, agent: CodeAgent) -> None:
    sleep(1.0)  # Let JavaScript animations happen before taking the screenshot
    driver = helium.get_driver()
    current_step = memory_step.step_number
    if driver is not None:
        for previous_memory_step in agent.memory.steps:  # Remove previous screenshots for lean processing
            if isinstance(previous_memory_step, ActionStep) and previous_memory_step.step_number <= current_step - 2:
                previous_memory_step.observations_images = None
        png_bytes = driver.get_screenshot_as_png()
        image = Image.open(BytesIO(png_bytes))
        print(f"Captured a browser screenshot: {image.size} pixels")
        memory_step.observations_images = [image.copy()]  # Create a copy to ensure it persists

    # Update observations with current URL
    url_info = f"Current url: {driver.current_url}"
    memory_step.observations = (
        url_info if memory_step.observations is None else memory_step.observations + "\n" + url_info
    )

In [11]:
from smolagents import HfApiModel

# Initialize the model
model_id = "meta-llama/Llama-3.3-70B-Instruct"  # You can change this to your preferred model ... THIS MODEL REQUIRES A PRO PRESCRIPTION!

# Lets try a smaller model shall we ... 
model_id = "meta-llama/Llama-3.2-3B-Instruct"

model = HfApiModel(model_id)

# Create the agent
agent = CodeAgent(
    tools=[go_back, close_popups, search_item_ctrl_f],
    model=model,
    additional_authorized_imports=["helium"],
    step_callbacks=[save_screenshot],
    max_steps=20,
    verbosity_level=2,
)


In [12]:
# Import helium for the agent
agent.python_executor("from helium import *", agent.state)

(None, '', False)

In [13]:
helium_instructions = """
You can use helium to access websites. Don't bother about the helium driver, it's already managed.
We've already ran "from helium import *"
Then you can go to pages!
Code:
```py
go_to('github.com/trending')
```<end_code>

You can directly click clickable elements by inputting the text that appears on them.
Code:
```py
click("Top products")
```<end_code>

If it's a link:
Code:
```py
click(Link("Top products"))
```<end_code>

If you try to interact with an element and it's not found, you'll get a LookupError.
In general stop your action after each button click to see what happens on your screenshot.
Never try to login in a page.

To scroll up or down, use scroll_down or scroll_up with as an argument the number of pixels to scroll from.
Code:
```py
scroll_down(num_pixels=1200) # This will scroll one viewport down
```<end_code>

When you have pop-ups with a cross icon to close, don't try to click the close icon by finding its element or targeting an 'X' element (this most often fails).
Just use your built-in tool `close_popups` to close them:
Code:
```py
close_popups()
```<end_code>

You can use .exists() to check for the existence of an element. For example:
Code:
```py
if Text('Accept cookies?').exists():
    click('I accept')
```<end_code>
"""

In [14]:
search_request = """
Please navigate to https://en.wikipedia.org/wiki/Chicago and give me a sentence containing the word "1992" that mentions a construction accident.
"""

agent_output = agent.run(search_request + helium_instructions)
print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Please navigate to https://en.wikipedia.org/wiki/Chicago and give me a sentence containing the word "1992" that │
│ mentions a construction accident.                                                                               │
│                                                                                                                 │
│ You can use helium to access websites. Don't bother about the helium driver, it's already managed.              │
│ We've already ran "from helium import *"                                                                        │
│ Then you can go to pages!                                                                                       │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ go_to('github.com/trending')                                                                                    │
│ ```<end_code>                                                                                                   │
│                                                                                                                 │
│ You can directly click clickable elements by inputting the text that appears on them.                           │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ click("Top products")                                                                                           │
│ ```<end_code>                                                                                                   │
│                                                                                                                 │
│ If it's a link:                                                                                                 │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ click(Link("Top products"))                                                                                     │
│ ```<end_code>                                                                                                   │
│                                                                                                                 │
│ If you try to interact with an element and it's not found, you'll get a LookupError.                            │
│ In general stop your action after each button click to see what happens on your screenshot.                     │
│ Never try to login in a page.                                                                                   │
│                                                                                                                 │
│ To scroll up or down, use scroll_down or scroll_up with as an argument the number of pixels to scroll from.     │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ scroll_down(num_pixels=1200) # This will scroll one viewport down                                               │
│ ```<end_code>                                                                                                   │
│                                                       

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Call:                                                                                                              
```py                                                                                                              
go_to('https://en.wikipedia.org/wiki/Chicago#Construction_accident')                                               
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  go_to('https://en.wikipedia.org/wiki/Chicago#Construction_accident')                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out: None

...

[Step 1: Duration 0.80 seconds| Input tokens: 4,982 | Output tokens: 46]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Captured a browser screenshot: (968, 894) pixels


[Step 2: Duration 0.00 seconds| Input tokens: 7,473 | Output tokens: 69]

TypeError: 'NoneType' object is not subscriptable

In [15]:
github_request = """
I'm trying to find how hard I have to work to get a repo in github.com/trending.
Can you navigate to the profile for the top author of the top trending repo, and give me their total number of commits over the last year?
"""

agent_output = agent.run(github_request + helium_instructions)
print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I'm trying to find how hard I have to work to get a repo in github.com/trending.                                │
│ Can you navigate to the profile for the top author of the top trending repo, and give me their total number of  │
│ commits over the last year?                                                                                     │
│                                                                                                                 │
│ You can use helium to access websites. Don't bother about the helium driver, it's already managed.              │
│ We've already ran "from helium import *"                                                                        │
│ Then you can go to pages!                                                                                       │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ go_to('github.com/trending')                                                                                    │
│ ```<end_code>                                                                                                   │
│                                                                                                                 │
│ You can directly click clickable elements by inputting the text that appears on them.                           │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ click("Top products")                                                                                           │
│ ```<end_code>                                                                                                   │
│                                                                                                                 │
│ If it's a link:                                                                                                 │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ click(Link("Top products"))                                                                                     │
│ ```<end_code>                                                                                                   │
│                                                                                                                 │
│ If you try to interact with an element and it's not found, you'll get a LookupError.                            │
│ In general stop your action after each button click to see what happens on your screenshot.                     │
│ Never try to login in a page.                                                                                   │
│                                                                                                                 │
│ To scroll up or down, use scroll_down or scroll_up with as an argument the number of pixels to scroll from.     │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ scroll_down(num_pixels=1200) # This will scroll one viewport down                                               │
│ ```<end_code>                                         

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
It seems like you're trying to get the number of commits made by the top author's fear of failure. Let's try again.
                                                                                                                   
We've already navigated to 'github.com/trending' and clicked on "Top products".                                    
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
go_to('github.com/trending')                                                                                       
click("Top products")                                                                                              
```                                                                                                                
We need to find the top author's profile.                                                                          
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
user = click("Top products").parent.parent.parent ultraclass="article-element")                                    
```                                                                                                                
Then we can get the author's information.                                                                          
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
author_username = user.find_by_text('User').text                                                                   
common_user = go_to("https://github.com/" + author_username+'s GitHub profile')                                    
```                                                                                                                
                                                                                                                   
This is internet isn't a shetik performers sims seems sink. The library is inside                                  
We need to follow the link to the top author's profile and get their GitHub information                            
                                                                                                                   
 Code:                                                                                                             
```py                                                                                                              
follow Link("https://github.com/u/)                                                                                
>"></gallery>                                                                                                      
if Link("https://github.com/").exists():                                                                           
while marginBottom aside of Poly:                                                                                  
author_info = click(Link("https://github.com/" + author_username)).text                                            
common_author = author_info.find_by_text('Commits')}   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  go_to('github.com/trending')                                                                                     
  click("Top products")                                                                                            
                                                                                                                   
  user = click("Top products").parent.parent.parent ultraclass="article-element")                                  
                                                                                                                   
  author_username = user.find_by_text('User').text                                                                 
  common_user = go_to("https://github.com/" + author_username+'s GitHub profile')                                  
                                                                                                                   
  follow Link("https://github.com/u/)                                                                              
  >"></gallery>                                                                                                    
  if Link("https://github.com/").exists():                                                                         
  while marginBottom aside of Poly:                                                                                
  author_info = click(Link("https://github.com/" + author_username)).text                                          
  common_author = author_info.find_by_text('Commits')}                                                             
  commits = common_author[1].count                                                                                 
  print(commits)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code parsing failed on line 4 due to: SyntaxError
user = click("Top products").parent.parent.parent ultraclass="article-element")                                    
^
Error: unmatched ')' (<unknown>, line 4)

...

[Step 1: Duration 0.76 seconds| Input tokens: 5,024 | Output tokens: 1,672]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Captured a browser screenshot: (968, 894) pixels


[Step 2: Duration 0.00 seconds| Input tokens: 7,536 | Output tokens: 2,508]

TypeError: 'NoneType' object is not subscriptable